# Stabilizer Code Simulator

*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

Quantum error correction is a set of techniques used to protect quantum information from errors caused by imperfect hardware or environmental noise. The main idea behind quantum error correction is to encode the quantum information in a way that errors can be detected and corrected without destroying the quantum state. This concept is crucial for the development of practical quantum computers, which require error correction to maintain the integrity of their quantum computations.

Stabilizer codes are a large family of quantum error correcting codes that include well-known codes such as the bit-flip code, phase-flip code, Shor code, Steane code, and surface code. In this tutorial, we introduce a systematic method for simulating general stabilizer codes on the [Baidu Quantum Platform](https://quantum.baidu.com/). This method enables users to study the effects of various types of noise, evaluate the performance of different error correction codes, and assess the robustness of quantum algorithms to errors.

## Theory

### Brief introduction

Stabilizer codes are originally introduced by Daniel Gottesman in 1997 and have become an important tool in the theory and practice of quantum error correction. The basic idea behind stabilizer codes is to encode a quantum state in such a way that errors can be detected and corrected. The key to this encoding is a set of operators called *stabilizers*, which are defined as the generators of a group of symmetries that leave the encoded state invariant. Specifically, the stabilizers are a set of commuting Pauli operators, which include the identity $I$, $X$, $Y$, and $Z$ operators.

More formally, a stabilizer code is defined by a pair of subspaces, the *code subspace* and the *complement subspace*, that are orthogonal complements of each other. The code subspace is spanned by the eigenstates of the stabilizers, which have eigenvalue $+1$. The complement subspace is spanned by the eigenstates of the stabilizers with eigenvalue $-1$. The stabilizers themselves are products of Pauli operators that act non-trivially only on the qubits in the code subspace.

To see how stabilizer codes protect quantum information, consider the case where a single qubit experiences an error due to environmental noise. This error can be represented by a Pauli operator that acts non-trivially on the affected qubit. If the code subspace is designed so that the stabilizers commute with this error operator, then the error can be detected by measuring the eigenvalues of the stabilizers. Specifically, if the eigenvalues of all the stabilizers are $+1$, then the error has not occurred. If the eigenvalues of one or more stabilizers are $-1$, then the error has occurred and the qubit must be corrected.

A stabilizer code can be mathematically described using three parameters,

$$
\llbracket n, k, d\rrbracket,
$$

where $n$ is the number of *physical qubits* forming the code subspace, $k$ is the number of *logical qubits* that can be protected from error, and $d$ is the *minimum distance* of the code. The minimum distance is the smallest number of simultaneous qubit errors that can transform one valid codeword into another or the minimum number of errors required to change one encoded state into another. The distance of the code measures its error-correcting capability, with higher distance values indicating better error correction. It's important to note that every useful stabilizer code must have $n > k$, as physical redundancy is necessary to detect and correct errors without disturbing the logical state. Let $\mathbb{P}_n$ be the $n$-qubit Pauli group and $\mathcal{S}\subseteq\mathbb{P}_n$ be the set of stabilizer generators corresponding to the stabilizer code $\llbracket n, k, d\rrbracket$. By definition, the number of stabilizer generators satisfies $r = \vert\mathcal{S}\vert = n − k$.

### Logical states

An $n$-qubit quantum state $\vert\psi\rangle$ is a valid *logical state* of a stabilizer code $\llbracket n, k, d\rrbracket$, also known as the *codeword state*, if and only if for every Pauli operator $P\in\mathcal{S}$, where $\mathcal{S}$ is the stabilizer generators of the code, it holds that $P\vert\psi\rangle=\vert\psi\rangle$. That is, the state $\vert\psi\rangle$ is stabilized by $P$. All valid codeword states form a code subspace of dimension $2^k$, allowing us to encode $k$ logical qubits. Usually, we identify the logical basis states of the code subspace.

The logical state is very important in quantum error correction because it encodes the information that is to be protected against errors. By measuring the stabilizer generators, it is possible to detect and correct errors that have occurred in the encoded qubits without directly measuring the state of the logical qubits themselves. This is important because measuring a quantum state can disturb its state, potentially introducing errors that defeat the purpose of error correction.

**Example.** Consider the simplest example of the three-qubit bit-flip code $\llbracket 3, 1, 1\rrbracket$. It encodes $1$ logical qubit to $3$ physical quibts and is completely specified by the stabilizer generators $\mathcal{S}=\{IZZ, ZZI\}$. One can easily verify that $\vert 000 \rangle$ and $\vert 111 \rangle$ are stabilized by $\mathcal{S}$ and thus the corresponding code space $\mathcal{C}$ has dimension $2$. The complement subspace can be classified as follows

$$
\begin{align}
\mathcal{C} &= {\rm span}\{\vert 000 \rangle, \vert 111 \rangle\}, \\
\mathcal{C}_0^\perp &= {\rm span}\{\vert 001 \rangle, \vert 110 \rangle\}, \\
\mathcal{C}_1^\perp &= {\rm span}\{\vert 010 \rangle, \vert 101 \rangle\}, \\
\mathcal{C}_2^\perp &= {\rm span}\{\vert 100 \rangle, \vert 011 \rangle\}.
\end{align}
$$

Notice that each single-qubit bit-flip errors $X_0$, $X_1$, and $X_2$ will rotate the code space to a unique complement space, i.e., for arbitrary quantum state $\vert\psi\rangle\in\mathcal{C}$, it holds that $X_i\vert\psi\rangle \in \mathcal{C}_i^\perp$. By measuring the two stabilizers $IZZ$ and $ZZI$, which will not change the logical basis states $\vert 000 \rangle$ and $\vert 111 \rangle$, we are able to detect which qubit has been flipped and then flip the corrupted qubit back to its correct state.

### General framework

![QEC](./figures/qec-framework-en.png "Figure 1: General Framework of Quantum Error Correction. ")

As illustrated in the above figure, the general framework of a stabilizer code involves four key steps: encoding, error detection, error correction, and decoding. Specifically,

+ **Encoding:** In the encoding step, the quantum information $\vert\psi\rangle$ is encoded onto a logical state in the code space that is protected by the stabilizer code. This is typically done using a set of quantum gates that implement a quantum error-correcting code. The encoded state is a superposition of logical basis states that are protected by the code.

+ **Error Detection:** In the error detection step, the stabilizers of the code are measured to check if any errors have occurred (known as the stabilizer measurement). The stabilizers are a set of Pauli operators that commute with each other and act non-trivially only on the code subspace. The eigenvalues of the stabilizers are measured to determine if any errors have occurred. If all stabilizers have eigenvalue $+1$, the state is in the code subspace and no errors have occurred. If any stabilizer has eigenvalue $-1$, then an error has occurred. Notice that in this step, we need clean ancilla qubits to implement the stabilizer measurements and record the error syndromes.

+ **Error Correction:** In the error correction step, the error is corrected by applying a set of gates that depend on the error syndrome, which is determined from the measurement of the stabilizers in the previous step. The gates are designed to reverse the effect of the error and return the state to the code subspace.

+ **Decoding:** In the decoding step, the encoded information $\vert\psi\rangle$ is extracted from the protected logical state. This is typically done by measuring a subset of the qubits in the code subspace, which collapses the state to one of the basis states. The measurement outcomes are used to reconstruct the original quantum state that was encoded using the code. In the simplest case, one can simply reverse the encoding procedure in step 1 to achieve decoding.

From the framework we can see that the key to the success of a stabilizer code is the choice of **stabilizers**, which must be carefully chosen to detect and correct errors while preserving the quantum information.

In summary, stabilizer codes are a powerful tool for protecting quantum information from decoherence due to environmental noise. They are defined by a set of commuting Pauli operators called stabilizers, which encode quantum states in a way that allows errors to be detected and corrected. Stabilizer codes have become an important tool in the theory and practice of quantum error correction, and have found numerous applications in quantum computing and communication.

## Practice

In this section, we will demonstrate step-by-step how to use the simulator implemented in QEP to simulate stabilizer codes. We will begin by showing how to instantiate the three-qubit bit-flip code and make use of its properties and methods. Next, we will use the well-known five qubit code to protect a single-qubit quantum state from arbitrary quantum noise.

### Three-qubit bit-flip code

The three-qubit bit-flip code is a quantum error correction code that can protect a single qubit against a single bit-flip error. It can be instantiated in QEP as follows.

First, we import the following necessary libraries:

In [ ]:
import copy

import QCompute
from Extensions.QuantumErrorProcessing.qcompute_qep.correction import BasicCode, ColorTable, FiveQubitCode
import Extensions.QuantumErrorProcessing.qcompute_qep.utils.circuit as circuit

To construct a stabilizer code, we need to specify its stabilizers and detectable error types. For the three-qubit bit-flip code, its stabilizers are {"IZZ", "ZZI"} and its detectable error type is the bit-flip error. In QEP, it can be instantiated as:

In [ ]:
bitflip_code = BasicCode(stabilizers=["IZZ", "ZZI"], error_types=["X"], name="Bit-Flip Code")

The stabilizer code has many useful methods. For example, you can check out its basic information

In [ ]:
print(bitflip_code)

You can access the encoding, detecting, correcting, and decoding quantum circuits of the stabilizer code in an extremely easy way

In [ ]:
bitflip_code.print_encode_circuit()  # Visualize the encoding circuit of the bit-flip code
bitflip_code.print_detect_circuit()  # Visualize the detecting circuit of the bit-flip code
bitflip_code.print_correct_circuit()  # Visualize the correcting circuit of the bit-flip code
bitflip_code.print_decode_circuit();  # Visualize the decoding circuit of the bit-flip code

You can also combine these methods together to see the composed quantum circuit

In [ ]:
bitflip_code.print_encode_decode_circuit()
bitflip_code.print_detect_correct_circuit()
bitflip_code.print_encode_detect_correct_decode_circuit();

### Five-qubit code

The five-qubit quantum error correction code is the smallest quantum error correcting code that can protect a logical qubit from any arbitrary single qubit error. It is originally invented by R. Laflamme, C. Miquel, J.P. Paz, and W.H. Zurek in 1996. For the five-qubit code, its stabilizers are {"XZZXI", "IXZZX", "XIXZZ", "ZXIXZ"} and its detectable error type is arbitrary single-qubit error. In QEP, the five-qubit code is a built-in quantum error correction code which you can obtain via

In [ ]:
fivequbit_code = FiveQubitCode()
print(fivequbit_code)

Alternatively, you can create this code in the old-fashioned style

In [ ]:
fivequbit_code = BasicCode(
    stabilizers=["XZZXI", "IXZZX", "XIXZZ", "ZXIXZ"], error_types=["X", "Y", "Z"], name="Five-Qubit Code"
)

We apply the five-qubit code to protect a single-qubit quantum state from arbitrary single-qubit quantum noise.
Before going to the details, we first define the following print function for later use.

In [ ]:
def print_counts(counts: dict, n: int, k: int):
    r"""Print measurement outcomes by splitting different roles."""
    for key, val in counts.items():
        # Print ancilla qubits
        print("'", end="")
        if n - k > 0:
            print("{}".format(ColorTable.ANCILLA + key[0 : n - k] + ColorTable.END), end="")
            print("{}".format(ColorTable.PHYSICAL + key[n - k : 2 * (n - k)] + ColorTable.END), end="")
        print("{}".format(ColorTable.ORIGINAL + key[2 * (n - k) : 2 * n - k] + ColorTable.END), end="")
        print("': {}".format(val))

Assume the single-qubit quantum state to be protected is prepared as

In [ ]:
qp = QCompute.QEnv()
qp.Q.createList(1)
QCompute.X(qp.Q[0])

Ideally, this quantum circuit produces the $\vert 1 \rangle$ state and has the form

In [ ]:
circuit.print_circuit(qp.circuit, colors={"red": [0]});

If we measure the circuit in the computational basis $8192$ shots, we must obtain $8192$ numbers of outcome $1$:

In [ ]:
raw_qp = copy.deepcopy(qp)
QCompute.MeasureZ(*raw_qp.Q.toListPair())
raw_qp.backend(QCompute.BackendName.LocalBaiduSim2)
result = raw_qp.commit(shots=8192, fetchMeasure=True)
print_counts(result["counts"], n=1, k=1)

Assume that this qubit suffers from the *completely depolarizing noise*, which will bring arbitrary input state to the maximally mixed quantum state. As a result, the quantum state is totally collapsed. If we measure the circuit in the computational basis, we will obtain outcomes $0$ and $1$ uniformly:

In [ ]:
corrupted_qp = copy.deepcopy(qp)
# The qubit suffers from the completely depolarizing noise
QCompute.ID(corrupted_qp.Q[0])
corrupted_qp.noise(gateNameList=["ID"], noiseList=[QCompute.Depolarizing(bits=1, probability=1)], qRegList=[0])
QCompute.MeasureZ(*corrupted_qp.Q.toListPair())
corrupted_qp.backend(QCompute.BackendName.LocalBaiduSim2)
result = corrupted_qp.commit(shots=8192, fetchMeasure=True)
print_counts(result["counts"], n=1, k=1)

We can use the five-qubit code to protect the target quantum state from the depolarizing noise

In [ ]:
# Step 1. Encode
enc_qp = fivequbit_code.encode(qp)
# Step 2. After encoding, the target qubit is damaged by the completely depolarizing noise
QCompute.ID(enc_qp.Q[0])
enc_qp.noise(gateNameList=["ID"], noiseList=[QCompute.Depolarizing(bits=1, probability=1)], qRegList=[0])
# Step 3. Detect and correct
cor_qp = fivequbit_code.detect_and_correct(enc_qp)
# Step 4. Decode
dec_qp = fivequbit_code.decode(cor_qp)

We measure the quantum error correction circuit in the computational basis $8192$ shots to see what will happen

In [ ]:
QCompute.MeasureZ(*dec_qp.Q.toListPair())
counts = circuit.execute(qp=dec_qp, qc=QCompute.BackendName.LocalBaiduSim2, shots=8192)
print_counts(counts, n=fivequbit_code.n, k=fivequbit_code.k)

Remarkably, it is observed that the target qubit (indicated in red) consistently yields the correct measurement outcome of $1$, despite being subject to complete depolarizing noise. This remarkable outcome highlights the efficacy of the five-qubit code, which can detect and rectify any single-qubit errors that may arise in the physical qubits. Conversely, the ancilla qubits (denoted in yellow) become destroyed, and their measurement outcomes, known as the "error syndromes," document the precise errors that have occurred.

## Summary

In this tutorial, we have briefly introduced the theory of stabilizer codes and its implementation on the [Baidu Quantum Platform](https://quantum.baidu.com/). We hope that this simulator can help users to study the effects of various types of noise, assess the performance of different error correction codes, and evaluate the robustness of quantum algorithms to errors.

## References

[1] Gottesman, Daniel. "Stabilizer Codes and Quantum Error Correction." PhD Thesis, **[California Institute of Technology](https://arxiv.org/abs/quant-ph/9705052)**, 1997.

[2] Nielsen, Michael A., and Isaac L. Chuang. "Quantum Computation and Quantum Information: 10th Anniversary Edition." **[Cambridge University Press](https://www.cambridge.org/highereducation/books/quantum-computation-and-quantum-information/01E10196D0A682A6AEFFEA52D53BE9AE#overview)**, 2010.

[3] Roffe, Joschka. "Quantum error correction: an introductory guide." **[Contemporary Physics](https://www.tandfonline.com/doi/abs/10.1080/00107514.2019.1667078)** 60.3 (2019): 226-245.

[4] Laflamme, Raymond, et al. "Perfect quantum error correcting code." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.77.198)** 77.1 (1996): 198.